In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install sdv

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd

from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

from sdv.sampling import Condition

In [ ]:
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/DLC-IDS/Datasets/train_merged_R2L.csv')
print(train_data.label.value_counts())

In [ ]:
attacks = ['R2L']

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_data)

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(train_data)

In [ ]:
# SYNTHETIC DATA GENERATION WITH CONDITION
generated_df = pd.DataFrame(columns=train_data.columns)

for feature in attacks:
  synthetic_data_condition = Condition(
      num_rows=150000,
      column_values={'label': feature}
  )

  synthetic_data = synthesizer.sample_from_conditions(
      conditions=[synthetic_data_condition]
  )

  print(synthetic_data.label.value_counts())
  generated_df = pd.concat([train_data, synthetic_data]) # Use pd.concat instead of generated_df.concat
  print(generated_df.label.value_counts())

In [ ]:
# saving the dataframe
generated_df.to_csv('/content/drive/MyDrive/DLC-IDS/Datasets/train_merged_R2L_sdv.csv', index = False)